## Loading Libraries 

In [533]:
#Importing Libraries

import simplejson as json 
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib
import numpy as np
import cv2
from PIL import Image, ImageStat 
import requests
import glob
import time
from datetime import date
import http.client, urllib.request 
import urllib.parse, urllib.error 
import base64, sys 
import pandas as pd


## Sign in Function

In [534]:
def signin(file='config.txt'):

    driver.get('https://www.linkedin.com/login')

    #Login credentials for Linkedin
    file=open(file)
    lines=file.readlines()
    username=lines[0]
    password=lines[1]

    # locate username and password form login page by HTML id
    enter_username = driver.find_element_by_id('username')
    enter_password=driver.find_element_by_id('password')

    # send_keys() to simulate key strokes
    enter_username.send_keys(username)
    enter_password.send_keys(password)

    #Submit Credentials
    enter_password.submit()


## Scroll Function

In [535]:
def scroll():
    SCROLL_PAUSE_TIME = 2
    
    #Scroll to see more and click if visible
    
    driver.execute_script("window.scrollTo(0,560);")
    time.sleep(SCROLL_PAUSE_TIME)
    
    try:
        driver.find_element_by_id('line-clamp-show-more-button').click()
    except:
        pass
    
    #Scroll back to top 
    
    driver.execute_script("window.scrollTo(0,0);")
    time.sleep(SCROLL_PAUSE_TIME)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")
    return soup


## Funcitions for Scraping 

In [536]:
def Scrape_name():
    name_all= soup.find('div',{'class':'flex-1 mr5'})#
    name=name_all.find('li',{'class':'inline t-24 t-black t-normal break-words'}).get_text().strip()
    return name

In [537]:
def Scrape_image():
    imgtag=soup.select('img[title]')
    urllib.request.urlretrieve(imgtag[0]['src'],'scraped_image.jpg')
    return imgtag[0]['src']


In [538]:
def Scrape_about():
    try:
        about_all= soup.find('span',{'class':'lt-line-clamp__raw-line'}).get_text().strip()
        return about_all
    except:
        return 'None'


In [539]:
def Scrape_header():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    header=name_all.find_all('h2')[0].get_text().strip()
    return header

In [540]:
def Scrape_location():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    location=name_all.find_all('li')[2].get_text().strip()
    return location

In [541]:
def Scrape_connections():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    connections=name_all.find_all('li')[3].get_text().strip()
    return connections

In [542]:
def Scrape_joining():
    company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
    joining=company_all.find_all('span')[2].get_text().strip()
    return joining

In [543]:
def Scrape_experience():
    company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
    experience=company_all.find_all('span')[4].get_text().strip()
    return experience

In [544]:
def Scrape_company():
    company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
    company=company_all.find_all('p')[1].get_text().strip().split('\n')[0]
    return company

In [545]:
def Scrape_title():
    company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
    title=company_all.find_all('h3')[0].get_text().strip()
    return title

In [546]:
def Scrape_college():
    college_all= soup.find('div',{'class':'pv-entity__degree-info'})
    college=college_all.find_all('h3')[0].get_text().strip()
    return college

In [547]:
def Scrape_degree():
    college_all= soup.find('div',{'class':'pv-entity__degree-info'})
    degree=college_all.find_all('span')[1].get_text().strip()
    return degree

In [548]:
def Scrape_degree_year():
    college_all= soup.find_all('time')
    year1=college_all[0].get_text().strip()
    year2=college_all[1].get_text().strip()
    year=year1 +'-'+ year2
    return year

## Login to Linkedin and enter profiles to scrape

In [549]:
df1=pd.read_csv('Link.csv')

dates=[]
name=[]
image=[]
about=[]
header=[]
location=[]
connections=[]
joining=[]
experience=[]
company=[]
title=[]
college=[]
degree=[]
degree_year=[]
# attributes for image analysis  
Brightness=[]
Face=[]
No_of_faces=[]
Face_area=[]
Bluriness=[]
# attributes for image score
img_score=[]
image_comments=[]

In [550]:
driver = webdriver.Chrome()
signin()
for link in df1.URL.values:
    Linkedin_url = link
    driver.get(Linkedin_url)
    soup=scroll()
    dates.append(pd.datetime.today().strftime('%d %b, %Y'))
    
    #Scrapping data and appending it to lists
    name.append(Scrape_name())
    image.append(Scrape_image())
    about.append(Scrape_about())
    header.append(Scrape_header())
    location.append(Scrape_location())
    connections.append(Scrape_connections())
    joining.append(Scrape_joining())
    experience.append(Scrape_experience())
    company.append(Scrape_company())
    title.append(Scrape_title())
    college.append(Scrape_college())
    degree.append(Scrape_degree())
    degree_year.append(Scrape_degree_year())

<ipython-input-550-dc7d285d88dc>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dates.append(pd.datetime.today().strftime('%d %b, %Y'))


## Image urls to jpg

In [551]:
# converting image urls to jpg files 
def url_to_jpg(i, url, file_path):
    # converting url to jpg format
    ext=".jpg"
    file_name = name[i]+ext.format(i) # photo saved under user's full name
    full_path = '{}{}'.format(file_path,file_name)
    urllib.request.urlretrieve(url,full_path)
    print('{} saved.'.format(file_name))
    return None
 
FILENAME = pd.DataFrame(image)
FILEPATH = '/Users/ancy_rex/imageanalytics/images/'

urls = FILENAME

for i, url in enumerate(urls.values):
    #calling the function for each user's link
    url_to_jpg(i, url[0], FILEPATH)

Ancy Rex.jpg saved.
Aarsha Treesa Joseph.jpg saved.
Sophia Binod.jpg saved.
Arpitha Tini Daniel.jpg saved.


## Image Analysis

In [552]:
def image_analysis(i):
    # path for each user image 
    ext=".jpg"
    file_name = name[i]+ext
    path = '/Users/ancy_rex/imageanalytics/images/'+file_name
    
    #opening the image using imread for BGR2GRAY conversion
    image  =  cv2.imread(path)
    gray_image  =  cv2.cvtColor(image,  cv2.COLOR_BGR2GRAY)

    #acquire total area of the image
    total_area = len(image)**2

    # obtain the number of faces in the image
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_image, 1.3, 5)

    # initialisation
    image_brightness = 0
    face = "Not Visible"
    no_of_faces = 0
    face_area = 0.0
    image_bluriness = 0.0
    
    # checking for only 1 face in the image
    if len(faces)==0 or len(faces)>1:
        image_brightness = 0
        face = "Not Visible"
        no_of_faces = 0
        face_area = 0
        image_bluriness = 0
        
    # calculating image attributes if image has 1 face
    elif len(faces)==1:
        no_of_faces = len(faces)
        face = "Visible"
        
        # converting image to luminous mode
        img  =  Image.open(path).convert('L')
        stat = ImageStat.Stat(img) #achieve each pixel brightness
        image_brightness  =  round(stat.mean[0]) #achieve average brightness
        
        for  (sx,sy,sw,sh) in faces: #finding area of the face 
            cv2.rectangle(image,(sx,sy),(sx+sw,sy+sh),(255,0,0),2)# drawing rectangle around the face
            crop_image = image[sy:sy+sh, sx:sx+sw] # the face part is cropped
            roi_gray = gray_image[sy:sy+sh, sx:sx+sw] 
            roi_color = image[sy:sy+sh, sx:sx+sw]
            face_area = round(sw*sh*100/total_area, 2) #area occupied by the face
            laplace  =  cv2.Laplacian(gray_image,  cv2.CV_64F).var()  #laplace transform
            image_bluriness = round(laplace, 2)
            
            face_gray  =  cv2.cvtColor(crop_image,  cv2.COLOR_BGR2GRAY)
            laplace  =  cv2.Laplacian(gray_image,  cv2.CV_64F).var()  #face laplace transform
            face_bluriness =	round(laplace, 2) #bluriness of the face
            
    #appending all image attribute values
    Brightness.append(image_brightness)
    Face.append(face)
    No_of_faces.append(no_of_faces)
    Face_area.append(face_area)
    Bluriness.append(image_bluriness)
    return(image)

## Image Scoring

In [553]:
#function for scoring profile photo
def image_score(brightness,face_area,bluriness,nooffaces):
    comments = []
    #initial photo score
    image_score = 15
    # scoring zero for not recognising the face or more than 1 recognised face
    if(nooffaces==0 or nooffaces>1):
        image_score = 0
        comments.append('Image should only contain your face')
    #scoring photo based on other attribute values and respective comments updated
    else:
        if(face_area < 35.0):
            image_score = image_score - 5
            comments.append('The image should accomodate more of your face')
        if(brightness < 100):
            image_score = image_score - 1
            comments.append('The Image requires more brightness')
        if(bluriness > 75):
            image_score = image_score - 1
            comments.append('The image is too blured')
    return(image_score,comments)

In [554]:
#function to clear the lists for junk/previous values
def clear_list():
    Brightness.clear()
    Face.clear()
    No_of_faces.clear()
    Face_area.clear()
    Bluriness.clear()
    img_scr.clear()
    img_comments.clear()
    

In [555]:
# clearing the list for new values 
clear_list()

# calling image analysis function for each profile photo
for i in range(0,len(df1)):
    
    img = image_analysis(i)
    
    # call image score function while passing attributes
    face_area=Face_area[i]
    brightness= Brightness[i]
    bluriness= Bluriness[i]
    nooffaces = No_of_faces[i]
    score,comment=image_score(brightness,face_area,bluriness,nooffaces)
    
    # appending score and comment for each user
    img_score.append(score)
    image_comments.append(comment)

## Write to csv

In [558]:
linkedin = {'date':dates,'name':name,'image':image,'about':about,'header':header,'location':location,'connections':connections,\
            'joining':joining,'experience':experience,'company':company,'title':title,'college':college,'degree':degree,\
            'degree_year':degree_year,'Image Score':img_score,'Image comments':image_comments}  
    
linkedin_df = pd.DataFrame(linkedin) 
linkedin_df.to_csv('Linkedin_profiles.csv')
linkedin_df

,date,name,image,about,header,location,connections,joining,experience,company,title,college,degree,degree_year,Image Score,Image comments
0,"16 Aug, 2020",Ancy Rex,https://media-exp1.licdn.com/dms/image/C5603AQ...,Master of Data Science student with highly dev...,Data Analyst Intern at AI Australia | Masters ...,"Melbourne, Victoria, Australia",159 connections,Employment Duration,Location,AI Australia,Data Analyst Intern,RMIT University,Master's degree,2019-2021,14,[The image is too blured]
1,"16 Aug, 2020",Aarsha Treesa Joseph,https://media-exp1.licdn.com/dms/image/C5603AQ...,None,Business Analytics Student at Deakin University,"Melbourne, Victoria, Australia",64 connections,Employment Duration,Location,Coles,Team member,Deakin University,Masters,2019-2020,9,[The image should accomodate more of your face...
2,"16 Aug, 2020",Sophia Binod,https://media-exp1.licdn.com/dms/image/C5603AQ...,None,"Student Business Analyst at Deakin University,...","Melbourne, Victoria, Australia",101 connections,Employment Duration,Location,Deakin University,Student Volunteer,Deakin University,Masters,2019-2020,0,[Image should only contain your face]
3,"16 Aug, 2020",Arpitha Tini Daniel,https://media-exp1.licdn.com/dms/image/C5103AQ...,None,Engineering Management student at Deakin Unive...,"Melbourne, Victoria, Australia",90 connections,Dec 2019 – May 2020,6 mos,UniLodge Australia,Residential Advisor,Deakin University,Master's degree,2019-2021,9,[The image should accomodate more of your face...
